# Calculator Applet

Demo that does various calculations - sha, hmac, pbkdf, field math, elliptic curve math

In [1]:
from applets.helpers import get_connection, AppletBase

class Calculator(AppletBase):
    def __init__(self, connection=None):
        super().__init__("B00B5111CC01", connection)

    def sha256(self, d):
        data = bytes([len(d)])+d
        r = "B0A10000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)
    
    def hmac_sha256(self, k, d):
        data = bytes([len(k)])+k+bytes([len(d)])+d
        r = "B0A20000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)
    
    def sha512(self, d):
        data = bytes([len(d)])+d
        r = "B0A30000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)
    
    def hmac_sha512(self, k, d):
        data = bytes([len(k)])+k+bytes([len(d)])+d
        r = "B0A40000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)

    def add_mod_FP(self, a, b):
        data = bytes([len(a)])+a+bytes([len(b)])+b
        r = "B0A50000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)
    
    def add_mod_N(self, a, b):
        data = bytes([len(a)])+a+bytes([len(b)])+b
        r = "B0A60000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)

In [2]:
connection = get_connection()
if connection is not None:
    atr=bytes(connection.getATR())
    print(atr.hex())
else:
    print("Failed to open a connection. No cardreaders?")

3bdc18ff8191fe1fc38073c821136605036351000250


In [3]:
app = Calculator(connection)
app.select()

b''

In [4]:
d = b'hello'
k = b'key'

In [5]:
# sha functions from the card
print("sha256:\t\t", app.sha256(d).hex())
print("hmac-sha256:\t", app.hmac_sha256(k, d).hex())
print("sha512:\t\t", app.sha512(d).hex())
print("hmac-sha512:\t", app.hmac_sha512(k, d).hex())

sha256:		 2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824
hmac-sha256:	 9307b3b915efb5171ff14d8cb55fbcc798c6c0ef1456d66ded1a6aa723a58b7b
sha512:		 9b71d224bd62f3785d96d46ad3ea3d73319bfbc2890caadae2dff72519673ca72323c3d99ba5c11d7c7acc6e14b8c5da0c4663475c2e5c3adef46f73bcdec043
hmac-sha512:	 ff06ab36757777815c008d32c8e14a705b4e7bf310351a06a23b612dc4c7433e7757d20525a5593b71020ea2ee162d2311b247e9855862b270122419652c0c92


In [6]:
import hashlib, hmac
# same using hashlib
print("sha256:\t\t",hashlib.sha256(d).digest().hex())
print("hmac-sha256:\t",hmac.new(k, d, digestmod=hashlib.sha256).digest().hex())
print("sha512:\t\t",hashlib.sha512(d).digest().hex())
print("hmac-sha512:\t",hmac.new(k, d, digestmod=hashlib.sha512).digest().hex())

sha256:		 2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824
hmac-sha256:	 9307b3b915efb5171ff14d8cb55fbcc798c6c0ef1456d66ded1a6aa723a58b7b
sha512:		 9b71d224bd62f3785d96d46ad3ea3d73319bfbc2890caadae2dff72519673ca72323c3d99ba5c11d7c7acc6e14b8c5da0c4663475c2e5c3adef46f73bcdec043
hmac-sha512:	 ff06ab36757777815c008d32c8e14a705b4e7bf310351a06a23b612dc4c7433e7757d20525a5593b71020ea2ee162d2311b247e9855862b270122419652c0c92


In [7]:
FP = 2**256 - 2**32 - 977
N = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141

In [8]:
def to_bytes(n:int):
    return n.to_bytes(32, 'big')

In [9]:
# random check
import random
for i in range(10):
    a = random.randint(1,N)
    b = random.randint(1,N)
    r = (a+b) % FP
    rr = r.to_bytes(32, 'big').hex()
    rc = app.add_mod_FP(to_bytes(a),to_bytes(b)).hex()
    print(i)
    if (rr!=rc):
        print(to_bytes(a).hex())
        print(to_bytes(b).hex())
        print(rr)
        print(rc)
        raise RuntimeError("Meh")

0
1
2
3
4
5
6
7
8
9


In [10]:
# edge cases - slightly less than mod, slightly more, equal and overflow
a = FP-20
for i, b in enumerate([10,23,20,10500, 1020203124]):
    r = (a+b) % FP
    rr = r.to_bytes(32, 'big').hex()
    rc = app.add_mod_FP(to_bytes(a),to_bytes(b)).hex()
    print(i)
    if (rr!=rc):
        print(to_bytes(a).hex())
        print(to_bytes(b).hex())
        print(rr)
        print(rc)
        raise RuntimeError("Meh")

0
1
2
3
4


In [11]:
from embit import bip32, ec
import os
seed = os.urandom(64)
root = bip32.HDKey.from_seed(seed)
# without prefix, depth, root fingerprint and index
root.serialize()[13:].hex()

'3aade779beb3befcec4b31b814cacccc94439133d762efc2df7c6ff49bff8a3300fe3c9db927ae4aba9cb38c696dc572690d7e3229f98163e5f35c64ff9c13bbb3'

In [12]:
res = root.child(10, hardened=False).serialize()[13:].hex()
res

'14e7c8ea8d7182bf76a75dec6f4a40e596252ca78221f9d86652348e95aa05480056e9b693be71f39ba59f2654457b4c828da85e8785c9a5ebb452af192c1713c0'

In [13]:
data = bytes([65])+root.serialize()[13:]+bytes([4,0x00,0,0,10])
r = "B0A90000"+(bytes([len(data)])+data).hex().upper()
res2 = app.request(r).hex()
if res2!=res:
    raise RuntimeError("Meh")
res2

'14e7c8ea8d7182bf76a75dec6f4a40e596252ca78221f9d86652348e95aa05480056e9b693be71f39ba59f2654457b4c828da85e8785c9a5ebb452af192c1713c0'

In [14]:
# point addition
p = root.to_public().key
p.compressed = False
print(p.sec().hex())
pp = ec.PublicKey.parse(p.serialize())
ec.secp256k1.ec_pubkey_tweak_add(pp._point, b'4'*32)
res = pp.sec().hex()
print(res)

04cfe79e2d251f371d488bfa038bf509223001244c060b56d03c43f464f0cf4d49d6a5fd144771082110c8106031ddb763793e99fddee16a616fa4adbc0442f4bb
04094d3886f993ba2dc5fe90c9f5effb15e436503747cdf3e202d5616cdd8c8a057f286e721f25673edce59788b57f9b7f364e5b6a7d051348b608a5cd763e5e94


In [15]:
data = bytes([32])+b'4'*32+bytes([65])+p.sec()
r = "B0A70000"+(bytes([len(data)])+data).hex().upper()
res2 = app.request(r).hex()
if res2!=res:
    raise RuntimeError("Meh")
res2

'04094d3886f993ba2dc5fe90c9f5effb15e436503747cdf3e202d5616cdd8c8a057f286e721f25673edce59788b57f9b7f364e5b6a7d051348b608a5cd763e5e94'

In [16]:
p3 = ec.PublicKey.parse(ec.secp256k1.ec_pubkey_serialize(ec.secp256k1.ec_pubkey_combine(p._point,pp._point)))
p3.compressed = False
res = p3.sec().hex()
res

'04ed5f5c06736ef6463f214c56fc437c7f2f88674bff2a4ce62cc680d87a8ce5dd2354748c33c10b8405f221c56a187ee3bb4ae22949121103b908304f9436b1a6'

In [17]:
data = bytes([65])+pp.sec()+bytes([65])+p.sec()
r = "B0A80000"+(bytes([len(data)])+data).hex().upper()
res2 = app.request(r).hex()
if res2!=res:
    raise RuntimeError("Meh")
res2

'04ed5f5c06736ef6463f214c56fc437c7f2f88674bff2a4ce62cc680d87a8ce5dd2354748c33c10b8405f221c56a187ee3bb4ae22949121103b908304f9436b1a6'

In [18]:
xpub = root.to_public()
xpub.serialize()[13:].hex()

'3aade779beb3befcec4b31b814cacccc94439133d762efc2df7c6ff49bff8a3303cfe79e2d251f371d488bfa038bf509223001244c060b56d03c43f464f0cf4d49'

In [19]:
res=xpub.child(10, hardened=False).serialize()[13:].hex()
res

'14e7c8ea8d7182bf76a75dec6f4a40e596252ca78221f9d86652348e95aa054802dc696a619ee02543161d1457611181c80dbc6c809eb06e281679eb424fabbfcf'

In [20]:
data = bytes([65])+xpub.serialize()[13:]+bytes([4,0x00,0,0,10])
r = "B0AA0000"+(bytes([len(data)])+data).hex().upper()
res2=app.request(r).hex()
if res2!=res:
    raise RuntimeError("Meh",res2)
res2

'14e7c8ea8d7182bf76a75dec6f4a40e596252ca78221f9d86652348e95aa054802dc696a619ee02543161d1457611181c80dbc6c809eb06e281679eb424fabbfcf'

In [21]:
import hashlib
print(hashlib.pbkdf2_hmac('sha512', b'pwd', b'salt', 2048, 64).hex())
print(hashlib.pbkdf2_hmac('sha512', b'pwd', b'salt', 100, 64).hex())

82218bfe339fa8a349423df85eaf89728b4e7dc02360e8c6feec346a8e46232db544f6f2d2fff1430e53f76c0adafd9a2087652b7b943fdc07c0cff4c74252e2
038c39d10762fd77de3a035e2c8c7ba4410f07f8251a28b1d670649abc256e810afb12e6c7e4b3e976fc1be1a387c345b282c9127f02b23862c16859bde9ff7f


In [22]:
import time
time.time()

1586742773.019443

In [23]:
t0 = time.time()
# 2048 iterations -> 0x8000
iterations = 100
data = bytes([2])+iterations.to_bytes(2,'big')+bytes([3])+b'pwd'+bytes([4])+b'salt'
r = "B0A00000"+(bytes([len(data)])+data).hex().upper()
# print(r)
print(app.request(r).hex())
dt = time.time()-t0
print(f"{dt} seconds for {iterations} iterations.\nFor 2048 iterations it will be {dt*2048/iterations} seconds")

038c39d10762fd77de3a035e2c8c7ba4410f07f8251a28b1d670649abc256e810afb12e6c7e4b3e976fc1be1a387c345b282c9127f02b23862c16859bde9ff7f
3.2807018756866455 seconds for 100 iterations.
For 2048 iterations it will be 67.1887744140625 seconds


In [24]:
import os
a = os.urandom(32)
pow(int.from_bytes(a,'big'),2,FP).to_bytes(32,'big').hex()

'738580a04282bfa066ebe55ae7fb8130ce5b30ac5f6a5ed836cd5067c8dffb39'

In [25]:
data = bytes([32])+a
r = "B0AB0000"+(bytes([len(data)])+data).hex().upper()
# print(r)
print(app.request(r).hex())


738580a04282bfa066ebe55ae7fb8130ce5b30ac5f6a5ed836cd5067c8dffb39


In [26]:
pow(int.from_bytes(a,'big'),3,FP).to_bytes(32,'big').hex()

'282f79cbbb62d09f9deced2d1b60d4c57d7b0ac1224e36b36cce44764dad437e'

In [27]:
data = bytes([32])+a
r = "B0AC0000"+(bytes([len(data)])+data).hex().upper()
# print(r)
print(app.request(r).hex())

282f79cbbb62d09f9deced2d1b60d4c57d7b0ac1224e36b36cce44764dad437e


In [28]:
pow(int.from_bytes(a,'big'),FP-2,FP).to_bytes(32,'big').hex()

'cba71ef1c2296ffb49aba3a82d01fd4ea1dc3f2d42fb8f7ead37a6f2f7385e23'

In [29]:
data = bytes([32])+a
r = "B0AD0000"+(bytes([len(data)])+data).hex().upper()
# print(r)
print(app.request(r).hex())

cba71ef1c2296ffb49aba3a82d01fd4ea1dc3f2d42fb8f7ead37a6f2f7385e23


In [30]:
pow(int.from_bytes(a,'big'),2,N).to_bytes(32,'big').hex()

'8ac8693f9ebed7ba82257263fb589ec80951cbf37e9a73940933563609f1f03a'

In [31]:
data = bytes([32])+a
r = "B0AE0000"+(bytes([len(data)])+data).hex().upper()
# print(r)
print(app.request(r).hex())


8ac8693f9ebed7ba82257263fb589ec80951cbf37e9a73940933563609f1f03a


In [32]:
pow(int.from_bytes(a,'big'),3,N).to_bytes(32,'big').hex()

'9472bf3e4a952f5ab87ad874a194722dfe91c017d8ddd64be3a527f1618583ef'

In [33]:
data = bytes([32])+a
r = "B0AF0000"+(bytes([len(data)])+data).hex().upper()
# print(r)
print(app.request(r).hex())


9472bf3e4a952f5ab87ad874a194722dfe91c017d8ddd64be3a527f1618583ef


In [34]:
pow(int.from_bytes(a,'big'),N-2,N).to_bytes(32,'big').hex()

'021fa509fb0338ddd0679de7e391669f98b0bd8bf0a317cce060f08e4f7d3ba3'

In [35]:
data = bytes([32])+a
r = "B0B00000"+(bytes([len(data)])+data).hex().upper()
# print(r)
print(app.request(r).hex())


021fa509fb0338ddd0679de7e391669f98b0bd8bf0a317cce060f08e4f7d3ba3


In [36]:
pub = ec.PrivateKey(os.urandom(32)).get_public_key()
sec = pub.sec()
pub.compressed=False
sec_unc = pub.sec()
sec_unc.hex()

'04705860902421a07293f32eb0576b2c70743fe9bffaf82eae8127b1b4f67cad0550cadead4bbe14baec3ea0a579efe525642ce12bbbacd973c07b58919c02029e'

In [37]:
data = bytes([65])+sec_unc
r = "B0B10000"+(bytes([len(data)])+data).hex().upper()
# print(r)
res = app.request(r)
print(res.hex())
res == sec

02705860902421a07293f32eb0576b2c70743fe9bffaf82eae8127b1b4f67cad05


True

In [38]:
data = bytes([33])+sec
r = "B0B20000"+(bytes([len(data)])+data).hex().upper()
# print(r)
res = app.request(r)
print(res.hex())
res == sec_unc

04705860902421a07293f32eb0576b2c70743fe9bffaf82eae8127b1b4f67cad0550cadead4bbe14baec3ea0a579efe525642ce12bbbacd973c07b58919c02029e


True

In [39]:
connection.disconnect()